In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
import base64
from dash.dependencies import Input, Output, State
#from IPython.display import display

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from CRUD import AnimalShelter


###########################
# Data Manipulation / Model
###########################


username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################


app = JupyterDash('SimpleExample')

# load client's logo
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    # unique identifier
    html.Div(id="query-out"),
    html.H1('Brendans Dash 12/18/2020'),
    html.Div(
        # radio buttons for interactive data filtering
        dcc.RadioItems(id='radio-id',
            options=[
                {'label': 'Water Rescue', 'value': 'water_rescue'},
                {'label': 'Mountain / Wilderness Rescue', 'value': 'mountain_rescue'},
                {'label': 'Disaster / Individual Tracking Rescue', 'value': 'disaster_rescue'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block'}
        ) 
    ),
    
    html.Hr(),
    # data table
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        # user-friendly features for interactive data table
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,        
    ),
    html.Br(),
    html.Hr(),
    
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display' : 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ]),
    # client's logo with hyperlink
    html.Div([
        html.A([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))            
        ], href='https://www.snhu.edu')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# callback for the radio buttons / interactive data filtering  
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('radio-id', 'value')])
def update_dashboard(input_value):
    # query logic for interactive data filtering (radio buttons)
    if input_value == 'water_rescue':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type":"Dog",
            "breed":
                {"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}
        }))
    elif input_value == 'mountain_rescue':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type":"Dog",
            "breed":
                {"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}
        }))
    elif input_value == 'disaster_rescue':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type":"Dog",
            "breed":
                {"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20},
            "age_upon_outcome_in_weeks":{"$lte":300}
        }))
    elif input_value == 'reset':
        df = pd.DataFrame.from_records(shelter.read({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')        
        
    return (data,columns)


# callback to update styles
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# callback for graph
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data')])
def update_graphs(viewData):

    # convert input to pandas data frame
    dff = pd.DataFrame.from_dict(viewData)
    display(dff)
    
    # values = calculate quantity of each animal color
    #values = pd.value_counts(df['color'], sort=False)      
    
    return [
        dcc.Graph(            
            figure = px.pie(dff, values=['dff.age_upon_outcome_in_weeks'],
                            names=['breed'],
                            title='Pie Chart of Animal Colors',
                            color_discrete_sequence=px.colors.sequential.RdBu),
        )    
    ]

# callback for geolocation map
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data')])
def update_map(viewData):
    df = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(df.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(df.iloc[0,9])
                ])
            ])
        ])
    ]



app